# 第1章: 基本的なプロンプト構造

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## セットアップ

次のセットアップセルを実行して、APIキーをロードし、`get_completion`ヘルパー関数を確立します。

In [ ]:
%pip install -qU pip
%pip install -qUr requirements.txt

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
from botocore.exceptions import ClientError
import json

# utilsパッケージからhintsモジュールをインポート
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None):
    # 推論設定を定義します
    inference_config = {
        "temperature": 0.0,  # 多様な応答を生成するための温度を設定します
        "maxTokens": 200,  # 生成するトークンの最大数を設定します
        "topP": 1,  # ニュークリアスamplingのためのtop_p値を設定します
    }
    # converseメソッドのパラメータを作成します
    converse_api_params = {
        "modelId": modelId,  # 使用するモデルIDを指定します
        "messages": [{"role": "user", "content": [{"text": prompt}]}],  # ユーザーのプロンプトを提供します
        "inferenceConfig": inference_config,  # 推論設定を渡します
    }
    # system_textが提供されているか確認します
    if system_prompt:
        # system_textが提供されている場合、converse_params辞書にシステムパラメータを追加します
        converse_api_params["system"] = [{"text": system_prompt}]

    # Bedrockクライアントにリクエストを送信して応答を生成します
    try:
        response = bedrock_client.converse(**converse_api_params)

        # 応答から生成されたテキストコンテンツを抽出します
        text_content = response['output']['message']['content'][0]['text']

        # 生成されたテキストコンテンツを返します
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"クライアントエラーが発生しました: {message}")

---

## レッスン

Amazon Bedrockは、Anthropic Claudeモデルと共に使用できる3つのAPIを提供しています。レガシーの[Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html)、[Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html)、および現在の[Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html)です。このチュートリアルでは、Converse APIのみを使用します。

最低限、Converse APIを使用してClaudeを呼び出すには、以下のパラメータが必要です：
- `modelId`: 呼び出すモデルの[APIモデル名](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns)

- `messages`: 入力メッセージの配列。Claude 3モデルは、交互に`user`と`assistant`の会話ターンで動作するように訓練されています。新しい`Message`を作成する際には、messagesパラメータで以前の会話ターンを指定し、モデルは次の会話の`Message`を生成します。
  - 各入力メッセージは、`role`と`content`を持つオブジェクトでなければなりません。単一の`user`ロールメッセージを指定することも、複数の`user`および`assistant`メッセージを含めることもできます（その場合は交互にする必要があります）。**最初のメッセージは常に`user`ロールを使用しなければなりません。**
  
  メッセージの内容は、[(ContentBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ContentBlock.html)の`content`フィールドに格納します。`text`フィールドにテキストを指定するか、モデルがサポートしている場合は、[(ImageBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ImageBlock.html)の`image`フィールドに画像の生バイトを渡すこともできます。ContentBlockの他のフィールドは[ツール使用](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html)のためのものです。

また、以下のようなオプションのパラメータもあります：
- `system`: システムプロンプト - これについては後述します。
  
- `temperature`: Claudeの応答の変動の度合い。このレッスンや演習では、`temperature`を0に設定しています。

- `max_tokens`: 停止する前に生成する最大トークン数。この最大値に達する前にClaudeが停止する場合があることに注意してください。このパラメータは、生成するトークンの絶対的な最大数を指定するだけです。さらに、これは*ハード*ストップであり、Claudeが単語の途中や文の途中で生成を停止する原因となる可能性があります。

すべてのAPIパラメータの完全なリストについては、[APIドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html)をご覧ください。

### 例

正しくフォーマットされたプロンプトに対するClaudeの応答を見てみましょう。以下の各セルについて、セルを実行してください（`shift+enter`）、するとClaudeの応答がブロックの下に表示されます。

In [ ]:
# プロンプト
PROMPT = "Hi Claude, how are you?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "Can you tell me the color of the ocean?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "What year was Celine Dion born in?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

正しいConverse APIフォーマットを含まないいくつかのプロンプトを見てみましょう。これらの不正なフォーマットのプロンプトに対して、Converse APIはエラーを返します。

まず、`messages`配列に`role`および`content`フィールドが欠けているConverse API呼び出しの例があります。

> ⚠️ **警告:** プロンプト内のmessagesパラメータのフォーマットが不正なため、以下のセルはエラーを返します。これは予期される動作です。

In [ ]:
# Claudeの応答を取得
inference_config = {
    "temperature": 0.0, # 多様な応答を生成するための温度を設定
    "maxTokens": 200 # 生成するトークンの最大数を設定
}

converse_api_params = {
    "modelId": modelId,
    "messages": [{"text":"Hi Claude, how are you?"}], # ユーザーのプロンプトを提供
    "inferenceConfig": inference_config # 推論設定を渡す
}

response = bedrock_client.converse(**converse_api_params)

# Claudeの応答を出力
print(response['output']['message']['content'][0]['text'])

役割の間で`user`と`assistant`を交互に切り替えないプロンプトがあります。

> ⚠️ **警告:** `user`と`assistant`の役割の交代がないため、Claudeはエラーメッセージを返します。これは予期される動作です。

In [ ]:
inference_config = {
    "temperature": 0.5,
    "maxTokens": 200
}
# converseメソッドのパラメータを作成する
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": [{"text": "Celine Dionは何年に生まれましたか？"}]},
        {"role": "user", "content": [{"text":"他に彼女についての事実を教えてもらえますか？"}]}
    ],
    "inferenceConfig": inference_config,
}

response = bedrock_client.converse(**converse_api_params)

# Claudeの応答を出力する
print(response['output']['message']['content'][0]['text'])

`user`と`assistant`のメッセージは**必ず交互に**なり、メッセージは**`user`のターンから始まる必要があります**。複数の`user`と`assistant`のペアをプロンプトに含めることができ（まるでマルチターンの会話をシミュレートするかのように）、`assistant`のメッセージの終わりに単語を入れて、Claudeがあなたの話の続きから進めることもできます（詳細は後の章で説明します）。

#### システムプロンプト

**システムプロンプト**も使用できます。システムプロンプトは、Claudeに質問やタスクを提示する前に**コンテキスト、指示、およびガイドラインを提供する方法**です。

構造的に、システムプロンプトは`user`と`assistant`のメッセージのリストとは別に存在し、したがって別の`system`パラメータに属します（ノートブックの[Setup](#setup)セクションにある`get_completion`ヘルパー関数の構造を見てください）。

このチュートリアル内では、システムプロンプトを利用する場合、あなたの完了関数に`system`フィールドを提供しています。システムプロンプトを使用したくない場合は、単に`SYSTEM_PROMPT`変数を空の文字列に設定してください。

#### システムプロンプトの例

In [ ]:
# SYSTEM_PROMPT = "あなたの答えは常に会話を進めるための批判的思考の質問のシリーズであるべきです（質問に対する答えを提供しないでください）。実際にユーザーの質問に答えないでください。"

# PROMPT = "なぜ空は青いのですか？"

# Claudeの応答を出力します
print(get_completion(PROMPT, SYSTEM_PROMPT))

なぜシステムプロンプトを使用するのか？**よく書かれたシステムプロンプトは、Claudeのパフォーマンスをさまざまな方法で向上させることができます。**たとえば、Claudeがルールや指示に従う能力を高めることができます。詳細については、Anthropicのドキュメントを訪れて、Claudeでの[システムプロンプトの使用方法](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)をご覧ください。

さて、いくつかの演習に入っていきましょう。上記のコンテンツを変更せずにレッスンプロンプトを試してみたい場合は、レッスンノートブックの一番下までスクロールして[**Example Playground**](#example-playground)にアクセスしてください。

---

## 演習
- [演習 1.1 - 三まで数える](#exercise-11---counting-to-three)
- [演習 1.2 - システムプロンプト](#exercise-12---system-prompt)

### 演習 1.1 - 三まで数える
適切な `user` / `assistant` フォーマットを使用して、以下の `PROMPT` を編集し、Claude に **三まで数えさせてください。** 出力は、あなたの解決策が正しいかどうかも示します。

In [ ]:
# Prompt - これは変更すべき唯一のフィールドです
PROMPT = "[Replace this text]"

# Claudeの応答を取得
response = get_completion(PROMPT)

# 演習の正確さを評価する関数
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Claudeの応答と対応する評価を出力
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_1_1_hint)

### 演習 1.2 - システムプロンプト

`SYSTEM_PROMPT`を修正して、Claudeが3歳の子供のように反応するようにします。

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[このテキストを置き換えて]"

# Prompt
PROMPT = "空はどれくらい大きいですか？"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- 採点 ---------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_1_2_hint)

### おめでとうございます！

ここまでのすべての演習を解決した場合、次の章に進む準備が整いました。楽しいプロンプト作成を！

---

## 例のプレイグラウンド

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのように影響するかを見るためのエリアです。

In [ ]:
# プロンプト
PROMPT = "Hi Claude, how are you?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "Can you tell me the color of the ocean?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "What year was Celine Dion born in?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# Claudeの応答を取得
inference_config = {
    "temperature": 0.0
}
additional_model_fields = {
    "max_tokens": 200
}

converse_api_params = {
    "modelId": modelId,
    "messages": [{"text":"Hi Claude, how are you?"}],
    "inferenceConfig": inference_config,
    "additionalModelRequestFields": additional_model_fields
}

response = bedrock_client.converse(**converse_api_params)

# Claudeの応答を出力
print(response['output']['message']['content'][0]['text'])

In [ ]:
inference_config = {
    "temperature": 0.0
}
additional_model_fields = {
    "top_p": 1,
    "max_tokens": 200
}
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": [{"text": "Celine Dionは何年に生まれましたか？"}]},
        {"role": "user", "content": [{"text":"他に彼女についての事実を教えてもらえますか？"}]}
    ],
    "inferenceConfig": inference_config,
    "additionalModelRequestFields": additional_model_fields
}

response = bedrock_client.converse(**converse_api_params)

# Claudeの応答を出力します
print(response['output']['message']['content'][0]['text'])

In [ ]:
# SYSTEM_PROMPT = "あなたの答えは常に会話を進めるための批判的思考の質問のシリーズであるべきです（質問に対する答えを提供しないでください）。実際にユーザーの質問に答えないでください。"

# PROMPT = "なぜ空は青いのですか？"

# Claudeの応答を出力します
print(get_completion(PROMPT, SYSTEM_PROMPT))